<a href="https://colab.research.google.com/github/Recollectionss/numeral-methods/blob/main/lab_3/lab_3_nm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np

BORDER = "-"*60
SPACE = " "*15

# Функція для системи рівнянь F(x) = 0
def F(x, funcs):
    return np.array([f(x) for f in funcs])

# Функція для обчислення похідної phi(x) за допомогою числового диференціювання
def derivative_phi(x, C, funcs, h=1e-5):
    phi_x = phi(x, C, funcs)
    phi_x_h = phi(x + h, C, funcs)
    return (np.linalg.norm(phi_x_h - phi_x)) / h


# Якобіан функцій F(x) (матриця часткових похідних)
def jacobian(x, funcs, num_vars):
    jacobian_matrix = np.zeros((num_vars, num_vars))
    h = 1e-5  # Малий крок для обчислення похідних (для числового диференціювання)
    for i in range(num_vars):
        x_h = np.copy(x)
        x_h[i] += h  # Збільшуємо i-у змінну на малий крок
        jacobian_matrix[:, i] = (F(x_h, funcs) - F(x, funcs)) / h
    print(f"Матриця Якобі для x = {x}: \n{jacobian_matrix}")
    return jacobian_matrix

# Перетворення Phi(x) = x - C * F(x), де C - невироджена матриця
def phi(x, C, funcs):
    return x - C @ F(x, funcs)

def simple_iteration_method(x0, funcs, tol=1e-6, max_iter=100):
    print(f"{BORDER} \n {SPACE}Метод простої ітерації")
    x = np.copy(x0)
    num_vars = len(x)
    iterations = 0

    # Початковий Якобіан для обчислення C
    C = np.linalg.inv(jacobian(x, funcs, num_vars))  # Матриця C
    print(f"Початковий Якобіан для обчислення C:\n {C}\n")

    # Обчислення максимального значення похідної phi(x) на проміжку [0, 0.5]
    q_max = max(derivative_phi(x, C, funcs,tol), derivative_phi(x + 0.5, C, funcs,tol))
    print(f"Максимальна похідна phi(x) на проміжку [0, 0.5]: {q_max} \n")

    for _ in range(max_iter):
        iterations += 1
        x_new = phi(x, C, funcs)  # Нова ітерація

        # Обчислення критерію зупинки
        stop_criterion = (q_max ** iterations) / (1 - q_max) * np.linalg.norm(x_new - x0)
        print(f"Ітерація {iterations + 1}: x = {x_new}")
        print(f"Критерій зупинки: {stop_criterion} \n")

        # Перевірка на точність за новим критерієм
        if stop_criterion <= tol:
            print(BORDER)
            return x_new

        x = x_new
    return None  # Якщо не знайдено розв'язок за max_iter ітерацій



# Якобіан функцій F(x) (матриця часткових похідних)
def jacobian(x, funcs, num_vars):
    jacobian_matrix = np.zeros((num_vars, num_vars))
    h = 1e-5  # Малий крок для обчислення похідних (для числового диференціювання)
    for i in range(num_vars):
        x_h = np.copy(x)
        x_h[i] += h  # Збільшуємо i-у змінну на малий крок
        jacobian_matrix[:, i] = (F(x_h, funcs) - F(x, funcs)) / h
    return jacobian_matrix

# Метод Гауса для розв'язання системи лінійних рівнянь Ax = b
def gauss_solve(A, b):
    n = len(b)
    # Прямий хід: приведення до трикутної форми
    for i in range(n):
        # Пошук максимального елемента у стовпці для часткового вибору головного елемента
        max_row = i + np.argmax(np.abs(A[i:, i]))
        # Перестановка рядків
        A[[i, max_row]] = A[[max_row, i]]
        b[[i, max_row]] = b[[max_row, i]]

        # Нормалізація поточного рядка
        for j in range(i + 1, n):
            factor = A[j, i] / A[i, i]
            A[j, i:] -= factor * A[i, i:]
            b[j] -= factor * b[i]

    # Зворотній хід: обчислення невідомих
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]
    return x

# Метод Ньютона
def newton_method(x0, funcs, tol=1e-6, max_iter=100):
    print(f"{BORDER} \n {SPACE}Метод Ньютона \n")
    x = np.copy(x0)
    num_vars = len(x)
    for iterations in range(max_iter):
        # Обчислення Якобіана в точці x
        J = jacobian(x, funcs, num_vars)
        # Обчислення значення функції F(x)
        Fx = F(x, funcs)

        # Розв'язання системи A_k z^k = F(x^k) методом Гауса
        z = gauss_solve(J, Fx)

        # Оновлення x: x^{k+1} = x^k - z^k
        x_new = x - z

        stop_criterion = np.linalg.norm(z)
        # Вивід результатів кожної ітерації
        print(f"Ітерація {iterations + 1}: x = {x_new}, ||z|| = {np.linalg.norm(z)}")
        print(f"Критерій зупинки: {stop_criterion} ")
        print(f"Матриця Якобі: \n {J}")
        print(f"Розв'язок СЛАР: {z}\n")

        # Перевірка умови зупинки
        if stop_criterion < tol:
            print(BORDER)
            return x_new

        x = x_new

    print("Метод Ньютона не збігся за вказану кількість ітерацій.")
    return None

# Приклад використання

# Кілька рівнянь для тесту
def f1(x):
    return 2 * x[0] - np.sin((x[0] - x[1]) / 2)

def f2(x):
    return 2 * x[1] - np.cos((x[0] + x[1]) / 2)

# Список функцій для системи рівнянь
funcs = [f1, f2]

# Початкове наближення
x0 = np.array([0, 0.5])

# Запускаємо методи
solution_simple_iteration = simple_iteration_method(x0, funcs, tol=1e-5)
# Запуск методу Ньютона
solution_newton = newton_method(x0, funcs, tol=1e-5)


# Вивід результатів
print("Розв'язок методом простої ітерації:", solution_simple_iteration)
# Вивід результатів
if solution_newton is not None:
    print("Розв'язок методом Ньютона:", solution_newton)
else:
    print("Метод Ньютона не зміг знайти розв'язок.")


------------------------------------------------------------ 
                Метод простої ітерації
Початковий Якобіан для обчислення C:
 [[ 0.67234823 -0.15337504]
 [-0.03916349  0.47980951]]

Максимальна похідна phi(x) на проміжку [0, 0.5]: 0.21873634285040827 

Ітерація 2: x = [-0.16157356  0.49477309]
Критерій зупинки: 0.04526064516414274 

Ітерація 3: x = [-0.16049344  0.49311303]
Критерій зупинки: 0.009837869133169775 

Ітерація 4: x = [-0.16051013  0.49310285]
Критерій зупинки: 0.002152128682562234 

Ітерація 5: x = [-0.16050991  0.49310232]
Критерій зупинки: 0.0004707481905066581 

Ітерація 6: x = [-0.16050991  0.49310231]
Критерій зупинки: 0.00010296974000478634 

Ітерація 7: x = [-0.16050991  0.49310231]
Критерій зупинки: 2.2523224348175885e-05 

Ітерація 8: x = [-0.16050991  0.49310231]
Критерій зупинки: 4.9266477231461166e-06 

------------------------------------------------------------
------------------------------------------------------------ 
                Метод Нь

In [ ]:
q_max = max(derivative_phi(x,C,funcs,tol))

def derivative_phi(x, C, funcs, h=1e-5):
    # Обчислюємо phi(x) для поточної точки
    phi_x = phi(x, C, funcs)

    # Обчислюємо phi(x + h) для кожної змінної окремо
    phi_x_h = np.array([phi(x + h * np.eye(len(x))[i], C, funcs) for i in range(len(x))])

    # Вираховуємо похідні
    derivatives = (phi_x_h - phi_x) / h

    # Повертаємо норму (для кожної змінної окремо)
    return np.linalg.norm(derivatives, axis=1)
